In [1]:
import teehr

In [3]:
# Create an Evaluation object to read from S3 bucket.
ev = teehr.Evaluation(dir_path="s3a://ciroh-rti-public-data/teehr-data-warehouse/v0_4_evaluations/e0_2_location_example")


In [4]:
ev.metrics.query(
    order_by=["primary_location_id"],
    group_by=["primary_location_id"],
    include_metrics=[
        teehr.Metrics.KlingGuptaEfficiency(),
        teehr.Metrics.NashSutcliffeEfficiency(),
        teehr.Metrics.RelativeBias()
    ]
).to_pandas()

24/12/30 19:31:27 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


,primary_location_id,kling_gupta_efficiency,nash_sutcliffe_efficiency,relative_bias
0,usgs-14138800,0.778544,0.560528,-0.050056
1,usgs-14316700,0.739875,0.568308,0.029096


In [5]:
ev.metrics.query(
    order_by=["primary_location_id", "month"],
    group_by=["primary_location_id", "month"],
    include_metrics=[
        teehr.Metrics.KlingGuptaEfficiency(),
        teehr.Metrics.NashSutcliffeEfficiency(),
        teehr.Metrics.RelativeBias()
    ]
).to_pandas()

,primary_location_id,month,kling_gupta_efficiency,nash_sutcliffe_efficiency,relative_bias
0,usgs-14138800,1,0.761423,0.509361,-0.020639
1,usgs-14138800,2,0.718381,0.567815,0.158800
2,usgs-14138800,3,0.606413,0.381857,0.240104
3,usgs-14138800,4,0.652380,0.325844,0.121470
4,usgs-14138800,5,0.713430,0.427701,-0.064748
5,usgs-14138800,6,0.480113,0.398414,-0.324427
6,usgs-14138800,7,0.110298,0.167976,0.020677
7,usgs-14138800,8,-0.365563,-0.025532,-0.107780
8,usgs-14138800,9,0.271357,0.255102,-0.395869
9,usgs-14138800,10,0.198704,0.361535,-0.609260


In [6]:
# Define a bootstrapper with custom parameters.
cb = teehr.Bootstrappers.CircularBlock(
    seed=50,
    reps=500,
    block_size=10,
    quantiles=[0.05, 0.95]
)
kge_cb = teehr.Metrics.KlingGuptaEfficiency(bootstrap=cb)
kge_cb.output_field_name = "kge_cb"

In [7]:
# Define a bootstrapper with custom parameters.
gum = teehr.Bootstrappers.Gumboot(
    seed=50,
    reps=500,
    quantiles=[0.05, 0.95]
)
kge_gum = teehr.Metrics.KlingGuptaEfficiency(bootstrap=gum)
kge_gum.output_field_name = "kge_gum"

In [8]:
include_metrics = [kge_cb, kge_gum, teehr.Metrics.KlingGuptaEfficiency()]

In [9]:
metrics_gdf = ev.metrics.query(
    include_metrics=include_metrics,
    group_by=["primary_location_id"],
    order_by=["primary_location_id"]
).to_pandas()
metrics_gdf

,primary_location_id,kge_cb,kge_gum,kling_gupta_efficiency
0,usgs-14138800,"{'kge_cb_0.05': 0.7506868839263916, 'kge_cb_0....","{'kge_gum_0.05': 0.7316278219223022, 'kge_gum_...",0.778544
1,usgs-14316700,"{'kge_cb_0.05': 0.6908299326896667, 'kge_cb_0....","{'kge_gum_0.05': 0.6515389084815979, 'kge_gum_...",0.739875


In [13]:
metrics_gdf.kge_cb.values

array([{'kge_cb_0.05': 0.7506868839263916, 'kge_cb_0.95': 0.7984060049057007},
       {'kge_cb_0.05': 0.6908299326896667, 'kge_cb_0.95': 0.7779679894447327}],
      dtype=object)

In [14]:
metrics_gdf.kge_gum.values

array([{'kge_gum_0.95': 0.8119508624076843, 'kge_gum_0.05': 0.7316278219223022},
       {'kge_gum_0.95': 0.7928256988525391, 'kge_gum_0.05': 0.6515389084815979}],
      dtype=object)

In [15]:
(
    ev.metrics.query(
        order_by=["primary_location_id", "year"],
        group_by=["primary_location_id", "year"],
        include_metrics=[
            metrics.Maximum(
                output_field_name="primary_annual_peak",
                input_field_names=["primary_value"]
            ),
            metrics.Maximum(
                output_field_name="secondary_annual_peak",
                input_field_names=["secondary_value"]
            ),
        ]
    )
    .to_pandas()
)

,primary_location_id,year,primary_annual_peak,secondary_annual_peak
0,usgs-14138800,2000,11.015253,1.430000
1,usgs-14138800,2001,16.452087,24.150000
2,usgs-14138800,2002,31.148531,29.109999
3,usgs-14138800,2003,21.719021,30.869999
4,usgs-14138800,2004,33.130711,30.099998
5,usgs-14138800,2005,22.030506,27.519999
6,usgs-14138800,2006,43.607944,53.669998
7,usgs-14138800,2007,28.316847,65.059998
8,usgs-14138800,2008,43.041607,29.689999
9,usgs-14138800,2009,37.378239,24.930000


In [16]:
cb = Bootstrappers.CircularBlock(
    seed=11,
    reps=100,
    block_size=1,
    quantiles=[0.05, 0.95]
)

df = (
    ev.metrics.query(
        order_by=["configuration_name", "primary_location_id", "year"],
        group_by=["configuration_name", "primary_location_id", "year"],
        include_metrics=[
            metrics.Maximum(
                output_field_name="primary_annual_peak",
                input_field_names=["primary_value"]
            ),
            metrics.Maximum(
                output_field_name="secondary_annual_peak",
                input_field_names=["secondary_value"]
            ),
        ]
    )
    .query(
        order_by=["configuration_name", "primary_location_id"],
        group_by=["configuration_name", "primary_location_id"],
        include_metrics=[
            metrics.RelativeBias(
                input_field_names=["primary_annual_peak", "secondary_annual_peak"],
                bootstrap=cb,
                output_field_name = "boot_ann_peak_rel_bias"
            )
        ]
    )
    .to_pandas()
)

In [17]:
df

,configuration_name,primary_location_id,boot_ann_peak_rel_bias
0,nwm30_retrospective,usgs-14138800,{'boot_ann_peak_rel_bias_0.05': -0.13140822947...
1,nwm30_retrospective,usgs-14316700,{'boot_ann_peak_rel_bias_0.05': -0.01148561574...


In [18]:
df.boot_ann_peak_rel_bias.values

array([{'boot_ann_peak_rel_bias_0.05': -0.131408229470253, 'boot_ann_peak_rel_bias_0.95': 0.27724918723106384},
       {'boot_ann_peak_rel_bias_0.05': -0.011485615745186806, 'boot_ann_peak_rel_bias_0.95': 0.4181682765483856}],
      dtype=object)

In [19]:
ev.attributes.to_pandas().head()

,name,type,description
0,2_year_flow,continuous,Two year flow rate in m^3/s
1,slope_mean,categorical,slope_mean
2,frac_urban,categorical,frac_urban
3,high_prec_freq,categorical,high_prec_freq
4,pet_mean,categorical,pet_mean


In [20]:
ev.locations.to_geopandas().head()

,id,name,geometry
0,usgs-14316700,"STEAMBOAT CREEK NEAR GLIDE, OR",POINT (-122.72894 43.34984)
1,usgs-14138800,"BLAZED ALDER CREEK NEAR RHODODENDRON, OR",POINT (-121.89147 45.45262)


In [21]:
ev.spark.stop()